In [ ]:
!lamin init --storage run-tests --modules bionty

In [ ]:
import warnings

import bionty as bt
import lamindb as ln
import pandas as pd
from spatialdata.datasets import blobs

warnings.filterwarnings("ignore")

ln.track()

In [ ]:
# Replace this with a proper dataset from for example https://spatialdata.scverse.org/en/latest/tutorials/notebooks/datasets/README.html

sdata = blobs()
sdata.attrs["sample"] = {
    "assay": "Visium Spatial Gene Expression",
    "disease": "Alzheimer's dementia",
    "developmental_stage": "very early",
}
sdata.tables["table"].var.index = [
    "ENSG00000139618",  # BRCA2
    "ENSG00000157764",  # BRAF
    "ENSG00000999999",  # Does not exist - to test add new
]
sdata.tables["table"].obs["sample_region"] = pd.Categorical(
    ["region 1"] * 13 + ["region 2"] * 13
)

In [ ]:
curator = ln.Curator.from_spatialdata(
    sdata,
    var_index={"table": bt.Gene.ensembl_gene_id},
    categoricals={
        "sample": {
            "assay": bt.ExperimentalFactor.name,
            "disease": bt.Disease.name,
            "developmental_stage": bt.DevelopmentalStage.name,
        },
        "table": {"sample_region": ln.ULabel.name},
    },
    organism="human",
)

curator.validate()
curator.add_new_from_var_index("table")
curator.add_new_from(key="developmental_stage", accessor="sample")
curator.add_new_from(key="sample_region", accessor="table")

curator.standardize(key="disease", accessor="sample")

artifact = curator.save_artifact(description="blob spatialdata")
artifact.describe()

In [ ]:
curator.save_artifact(description="blob spatialdata", revises=artifact)